In [1]:
#read in cassis 
#read in the json:
import json
import base64
import re
import sys
import os

from typing import Generator

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

PATH="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations"

ALLEN_NLP_PATH=os.path.join(  PATH, "bert-base-srl-2019.06.17.tar.gz" )
SPACY_PATH=os.path.join( PATH , "spacy-textcat" )

TEMPLATE_PATH=os.path.join( PATH, "out.html.template" )


sys.path.append( os.path.join( PATH ) )

with open( os.path.join( PATH , "tests/test_files/typesystems/typesystem.xml" )  , 'rb') as f:
    typesystem = load_typesystem(f)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/minus_lesser_of_response.json')) as json_file:
#    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/small_nested_tables_response.json')) as json_file:
#    response = json.load(json_file)
    
with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.json')) as json_file:
    response = json.load(json_file)  
    
#with open(  os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.json')) as json_file:
#    response = json.load(json_file)
    
OUTPUT_PATH=os.path.join( PATH, "tests/test_files/output_reporting_obligations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.html" )
    
decoded_cas=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas=load_cas_from_xmi( decoded_cas, typesystem=typesystem  )


In [2]:
from src.transform import ListTransformer
from src.reporting_obligations import ReportingObligationsFinder

transformer=ListTransformer( cas )

transformer.add_reporting_obligation_view( OldSofaID='html2textView', NewSofaID = 'ReportingObligationView'  )

sentences=cas.get_view( "ReportingObligationView" ).sofa_string

reporting_obligations_finder = ReportingObligationsFinder( sentences = sentences.split( "\n" ) )
list_xml=reporting_obligations_finder.process_sentences( ALLEN_NLP_PATH, SPACY_PATH )
reporting_obligations_finder.print_to_html( list_xml, TEMPLATE_PATH, OUTPUT_PATH )


loading AllenNLP predictor from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/bert-base-srl-2019.06.17.tar.gz
loading spacy model from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/spacy-textcat
ARGS[to] ARG2 to the Board and to the Court of Justice of the European Union
ARGS[] ARG1 an authority for the purposes of verifying the authenticity of individual contribution decisions
Writing output to /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/output_reporting_obligations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.html using /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/out.html.template as html template


In [4]:
a=['some', 'more'  ]
a+=None

TypeError: 'NoneType' object is not iterable

In [5]:
a+None

TypeError: can only concatenate list (not "NoneType") to list

In [8]:
sentence="something some more ❮  then then   ❯  dfdfgd ❮  then then   ❯ and more"

sentence = sentence.rstrip('\r\n')
subsentence = re.sub(r'(^[^❮]+|[^❯]+$)',r'', sentence)  #finds everything between " ❮ ❯ " ==>the main sentence
if len(subsentence) > 0: sentence = sentence.replace(subsentence, '', 1)  #remove everything inside " ❮ ❯ " from the string
    

In [13]:
subsentence=" capital instruments ‖ and/or tier ‖ 2 cap"

list_subsentences = re.sub(r' ‖ and/or ', r' and/or ', subsentence ).lstrip('❮').rstrip('❯').split(' ‖ ')
list_subsentences

[' capital instruments and/or tier', '2 cap']

#### END

In [2]:
from src.utils import SeekableIterator
from src.transform import get_other_lines, fold_sublist, transform_lines

value_between_tagtype_generator=cas.get_view( 'html2textView' ).select( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"  )        

seek_vbtt=SeekableIterator( value_between_tagtype_generator )

lines=get_other_lines( cas, 'html2textView' , seek_vbtt, 'root'    )

with open( os.path.join( PATH, "tests/test_files/sofa_reporting_obligations/minus_lesser_of_response.txt")  , "w" ) as f:
    f.write( "\n".join(transform_lines( lines )))

In [5]:
open(os.path.join( PATH, "tests/test_files/sofa_reporting_obligations/minus_lesser_of.txt") , 'r'   ).read()

'ANNEX I\nFormulae for the determination of the liquidity buffer composition. Credit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.\nFormulae for the determination of the liquidity buffer composition. Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭.\nFormulae for the determination of the liquidity buffer composition. ‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets, exc

In [3]:
fold_sublist(  [ '1.', 'b' , 'c'  ]  )

'❬b ‖ and/or c❭'

In [19]:
fold_sublist(  [ '1.', '(a) something' , 'c'  ]  )

'❬something ‖ and/or c❭'

In [15]:
fold_sublist(  [ '  1. ', 'something (a)' , [  'd', ['e','f'] ]  , 'c'  ]  )

'❬something (a) ‖ and/or d ⟨e ⟨f⟩⟩ ‖ and/or c❭'

In [ ]:
fold_sublist(  [ '  1. ', 'something (a)' , [  'd', ['e'] ]  , 'c'  ]  )

In [12]:
fold_sublist(  [ '  1. some ', 'something (a)' , 'c'  ]  )

'1. some ❬something (a) ‖ and/or c❭'

In [17]:
fold_sublist(  [ '  1.'  ]  )

'❬❭'

In [3]:
import pickle
with open(  os.path.join( PATH, "tests/test_files/pickle/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_nested_lines.p"), 'wb')  as f:
    # store the data as binary data stream
    pickle.dump(lines, f)

In [9]:

with open(  os.path.join( PATH, "tests/test_files/pickle/small_nested_tables_nested_lines.p"), 'rb') as filehandle:
    # read the data as binary data stream
    lines_test = pickle.load(filehandle)

In [2]:
from transform import ListTransformer

In [3]:
transformer=ListTransformer( cas )

In [4]:
transformer.add_reporting_obligation_view( OldSofaID='html2textView' )

In [ ]:
print( cas.get_view( "ReportingObligationView" ).sofa_string )

['5.   Competent authorities may waive the condition set out in point (d) of paragraph 4 where Article 20(1)(b) is applied. In that case additional objective criteria as set out in the delegated act referred to in Article 460 have to be met. Where such higher inflow is permitted to be applied, the competent authorities shall inform EBA about the result of the process referred to in Article 20(1)(b). Fulfilment of the conditions for such higher inflows shall be regularly reviewed by the competent authorities.',
 '6.   EBA shall develop draft regulatory technical standards to further specify the additional objective criteria referred to in paragraph 5.',
 'EBA shall submit those draft regulatory technical standards to the Commission by 1 January 2015.',
 'Power is delegated to the Commission to adopt the regulatory technical standards referred to in the first subparagraph in accordance with Articles 10 to 14 of Regulation (EU) No 1093/2010.',
 '7.   Institutions shall not report inflows 

In [2]:
#section_regexp = r'^(Part|Title|Chapter|Section|Sub-Section|Article) ([0-9.]|ONE|TWO|THREE|FOUR|FIVE|SIX|SEVEN|EIGHT|NINE|TEN|ELEVEN|TWELVE|THIRTEEN|FOURTEEN|FIFTEEN|SIXTEEN|SEVENTEEN|EIGHTEEN|NINETEEN|TWENTY|I|II|III|IV|V|VI|VII|VIII|IX|X|X(I|II|III|IV|V|VI|VII|VIII|IX|X))+ *$'

In [3]:
#paragraphs=cas.get_view( 'html2textView'   ).select( 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph')
#for par in paragraphs:
#    print( par.get_covered_text() )
#    print( "_"*100 )
#    print( "\n" )

In [5]:
class SeekableIterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.current = None
        self.reuse = False

    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()

    def next(self):
        if self.reuse:
            self.reuse = False
        else:
            self.current = None
            self.current = next(self.iterator)
        return self.current

    def rewind(self):
        self.reuse = True

In [6]:
def get_other_lines( cas: Cas, SofaID: str , value_between_tagtype_seekable_generator: Generator, root_paragraph, end=-1 ) -> list: 
    
    '''
    Convert cas with paragraph annotations to a nested lists to be used for annotation of reporting obligations.
    Structure of the nested lists is as follows:
    ->p   new_root (i.e. start of enumeration, e.g. "...shall be allocated as follows:"  )
    ->p
    ->p
    [->p  new_root
    [->p  new_root
    ->p]]
    '''

    lines=[]
    paragraphs_covering_tag=None
    new_root_paragraph=None
    
    for tag in value_between_tagtype_seekable_generator:
        
        #check if tag is a "p", and if it was not part of one of the already nested p's.
        if not tag.tagName == 'p':
            continue
        
        if end >0 and tag.begin > end:
            value_between_tagtype_seekable_generator.rewind()
            return lines
              
        paragraphs_covering_tag= ['root'] + list(cas.get_view( SofaID ).select_covering( "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph", tag ))
     
        #If no enumeration detected yet, set root_paragraph to 'root'.
        if root_paragraph=='root': 
            index_root=0
        else:
            index_root=get_index_typesystem_elements_in_list( paragraphs_covering_tag[1:], root_paragraph.xmiID )+1
        
        #sanity check
        assert index_root >= 0

        #Start of new sublist detected (i.e. the root_paragraph is no longer the "deepest" paragraph). Always go for the deepest root.
        if paragraphs_covering_tag[ index_root+1: ]:
            new_root_paragraph=paragraphs_covering_tag[ index_root+1 ]
            value_between_tagtype_seekable_generator.rewind()
            sublines=get_other_lines( cas, SofaID, value_between_tagtype_seekable_generator, new_root_paragraph, end=new_root_paragraph.end  )
            lines.append( sublines )
            
        else:
            lines.append(  tag.get_covered_text() )
            
    return lines

#helper function
def get_index_typesystem_elements_in_list( list_typesystem_elements: list, xmiID: int  ) -> int:
    
    '''
    Helper function to get index of certain typesystem element (with given xmiID) in list of typesystem elements
    Necessary because list_typesystem_elements.index( element ) does not return correct index,
    because all elements in list_typesystem_elements are considered equal (due to bug in dkpro-cassis)
    '''
    
    for index, item in enumerate(  list_typesystem_elements ):
        if item.xmiID == xmiID:
            return index
        
    return -1

In [7]:
value_between_tagtype_generator=cas.get_view( 'html2textView' ).select( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType")        

seek_vbtt=SeekableIterator( value_between_tagtype_generator )

lines=get_other_lines( cas , 'html2textView' , seek_vbtt , 'root' )

lines

['ANNEX I',
 ['\r\n Formulae for the determination of the liquidity buffer composition\r\n',
  '1.',
  'Credit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.',
  '2.',
  ['Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to:',
   '(a)',
   'the level 1 asset amount; plus',
   '(b)',
   'the level 2A asset amount; plus',
   '(c)',
   'the level 2B asset amount;',
   'minus the lesser of:',
   '(d)',
   'the sum of (a), (b), and (c); or',
   '(e)',
   'the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex.'],
  '3.',
  ['‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein:',
   '(a)',
   'an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets, excluding level&nbsp;1 c

In [8]:
def fold_sublist( sublist: list, open_paren='❬', close_paren='❭'  )-> str: 
    
    start_enumeration=''
    
    other_lines=[]
    
    for i,item in enumerate(sublist):
        
        if type(item)==str:
            item=clean_line(item).strip()
            if not item:
                continue
        
        if type(item)==str and i==0: #enumeration starts in next item
            
            start_enumeration = item
                        
        elif type(item)==str: #actual enumeration
            
            other_lines.append( item )
            
        elif type(item)==list:
            
            other_lines.append( fold_sublist(item, open_paren='⟨', close_paren='⟩'  ) )
                
    return (start_enumeration + ' '  + open_paren +  ' ‖ and/or '.join( other_lines ) + close_paren).strip()


def check_valid_list(  line: list ) -> bool:
    
    if len( line )<=1:
        return False
    
    if type( line[0])!=str:
        return False
    
    return True

def clean_line(  line: str ) -> str:

    line = re.sub(r'^( *\(([0-9]+|[a-z]|x?v?[i]+v?x?|xv?|v)\)|-|—|•)',r'',line)
    line = re.sub(r'^( *([0-9]+[.])) *$',r'',line)  #delete strings like "1. " ==> can occur inside nested list, and mess up function handle_root_lists
    line = re.sub(r'[;.,] *(or|and)? *$',r'', line)
    line = re.sub(r'\s*[:]\s*$', ' ', line)
    
    return line

def break_long_paragraphs(text: str ) -> list:
    if len(text) > 1000:
        text = re.sub(r'([^❮❬‖❭❯]{1000}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)
    
    break_lines = [line.strip() for line in text.split( "\n" ) if line.strip()]

    return break_lines


In [9]:
line='(a) dsf '
line=re.sub(r'^( *\(([0-9]+|[a-z]|x?v?[i]+v?x?|xv?|v)\)|-|—|•)',r'',line)
line

' dsf '

In [10]:
line='1.'
line=re.sub(r'^( *([0-9]+[.])) *$',r'',line)
line

''

In [11]:
def handle_root_list(main_sentences, last_sentence, other_lines):

    # default choice of action
    list_action = 'ADD_IN_SENTENCE'
    list_comma_handling = 'AUTO'

    # detect how to treat the list thanks to the last sentence
    if re.search(r' shall do the following $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
        last_sentence = re.sub(r' shall do the following $', r' shall ', last_sentence)
    if re.search(r' and do the following $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
        last_sentence = re.sub(r' and do the following $', r' and ', last_sentence)
    elif re.match(r'The following( definitions?|requirements)? shall apply ', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
        main_sentences.append(last_sentence+'.')
        last_sentence = ''
    elif re.search(r' the following( definitions?)? shall apply ', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'COMMA'
        last_sentence = re.sub(r' ?,? the following( definitions?)? shall apply $', r' ', last_sentence)
    elif re.search(r' the following requirements? shall apply ', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'COMMA'
        last_sentence = re.sub(r' ?,? the following( requirements?)? shall apply $', r' ', last_sentence)
    elif re.search(r' the following requirements ', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
        main_sentences.append(last_sentence+'.')
        last_sentence = ''
    elif re.search(r'\b[Tt]he( .*)? standards relating to .* are the following ', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
        main_sentences.append(last_sentence+'.')
        last_sentence = ''
    elif re.search(r' following ', last_sentence):
        list_action = 'KEEP_OUT'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' as follows $', last_sentence):
        list_action = 'KEEP_OUT'
        list_comma_handling = 'NO_COMMA'
        last_sentence = re.sub(r' as follows $',r' as described in the following list ', last_sentence)
    elif re.search(r' provided $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' (including|excluding|include|exclude|concerning) $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' (shall|can|must|should)( not)? $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' (that|which|who) $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' (when|where|in the cases?)( it (has|does))? $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' (of|on|by|for|from|to|using) $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' (more|less) than $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' (is|are|specify|specifies|comprise|comprises|mean|means) $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r' (report|disclose|collect|gather|store) $', last_sentence):
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'NO_COMMA'
    elif re.search(r'^ *(For\b|When|Where)[^,]+$', last_sentence): #word break added after For
        list_action = 'ADD_IN_SENTENCE'
        list_comma_handling = 'COMMA'
    elif re.search(r' points? \( ?(a|i) ?\)', main_line) and re.search(r' points? \( ?(a|i) ?\)', re.sub(r'(for the purposes? of( the)? points?|points? \([^)]+\)( (and|to) \([^)]+\))? ?of )', r'', re.sub(r' ?of this paragraph',r'',main_line, re.IGNORECASE), re.IGNORECASE)):
        list_action = 'KEEP_OUT'
        list_comma_handling = 'NO_COMMA'

        
    # detect whether to add a comma at the end of not
    if list_action == 'ADD_IN_SENTENCE' and list_comma_handling == 'AUTO':
        # TODO: list_comma_handling == AUTO
        list_comma_handling = 'NO_COMMA'

    # duplicate the last sentence as many times as required by the list
    sentence_spacer = " "
    if list_action == 'ADD_IN_SENTENCE':
        # output one per line if there's no shared context
        if len(main_sentences) == 0:
            sentence_spacer = '\n'
        elif len(main_sentences) == 1 and re.match('[0-9]+[.]', main_sentences[0]):
            sentence_spacer = '\n'
            last_sentence = main_sentences[0].rstrip() + ' ' + last_sentence
            main_sentences = []
        # also output one per line if we end up in a case where we wanted each list item to be its own sentence
        elif len(last_sentence) == 0: 
            main_sentences = [ ' '.join(map(lambda s: str(s).rstrip(), main_sentences)) ]
            sentence_spacer = '\n'
        # also output one per line if merging would result in a too long paragraph
        elif len(other_lines) > 9:
            sentence_spacer = '\n'
            last_sentence = ' '.join(map(lambda s: str(s).rstrip(), main_sentences)) + ' ' + last_sentence
            main_sentences = []
        # also output one per line if merging would result in a too long paragraph
        elif len(last_sentence) * len(other_lines) >= 200: 
            main_sentences = [ ' '.join(map(lambda s: str(s).rstrip(), main_sentences)) ]
            sentence_spacer = '\n'
        # deal with the comma, if necessary
        if list_comma_handling == 'COMMA':
            last_sentence = re.sub(r' *$', r', ', last_sentence)
        elif list_comma_handling != 'NO_COMMA':
            raise ValueError("Invalid list comma handling: " + str(list_comma_handling))
        # generate the filled-in sentences
        for other_line in other_lines:
            main_sentences.append(last_sentence + other_line + '.')
    elif list_action == 'KEEP_OUT':
        # append the options between brackets
        main_sentences.append(last_sentence.rstrip() + ' ❮' + ' ‖ '.join(other_lines) + '❯')
    else:
        raise ValueError("Invalid list action: " + str(list_action))

    if sentence_spacer == '\n':
        def convert_to_strong_list_if_possible(sentence):
            if '❬' in sentence:
                sentence_other = re.sub(r'(^[^❬]+|[^❭]+$)',r'',sentence)
                if len(sentence_other) > 0 and not ('❬' in sentence_other): 
                    sentence = sentence.replace(sentence_other, '', 1)
                    sentence = re.sub(r'\s*[:.;]\s*$', ' ', sentence)
                    sentence_other = sentence_other[1:len(sentence_other)-1]
                    return handle_root_list([], sentence, sentence_other.split(' ‖ and/or '))
                else:
                    return sentence
            else:
                return sentence
        main_sentences = list(map(convert_to_strong_list_if_possible, main_sentences))

    return sentence_spacer.join(main_sentences)

In [12]:
import string
string.punctuation
from spacy.lang.en import English 


def merge_sentences( lines: list  ) -> list:

    sentencizer = English()
    sentencizer.add_pipe(sentencizer.create_pipe('sentencizer'))

    other_lines=[]
    merged_sentences=[]

    for line in lines:

        main_line_other_line=()

        if type(line)==str:

            main_line_other_line=( line, None  )  #no other lines (i.e. no enumeration/detected lists)

        elif type(line)==list:

            #an enumeration should start with a string (i.e. type( line[0] )==str ), and should sum up at least one item (len( line )>1  ).  
            if not check_valid_list( line ):
                continue

            main_line=line[0]

            other_lines=[]
            for enumeration in line[1:]:
                if type(enumeration)==list:
                    other_lines.append(fold_sublist(enumeration))
                elif type(enumeration)==str:
                    enumeration=clean_line(enumeration).strip()
                    if enumeration:
                        other_lines.append( enumeration )

            main_line_other_line=( main_line, other_lines  )

        else:
            continue

        #handle root lists
        main_line =main_line_other_line[0].strip()
        if not main_line.strip():
            continue

        if not main_line_other_line[1]: #i.e. no enumeration
            #Maybe should also handle processing of sections
            merged_sentences+=break_long_paragraphs( main_line )
            continue

        #Handle main lines (i.e. start of enumerations) that do not end with a punctuation. 
        if main_line.strip()[-1] not in string.punctuation:
            main_line=main_line+'.'

        main_sentences = list(map(lambda s: re.sub(r'[)](?![ ,;:.?! ()])(?!th)', r') ',s.text.rstrip()), sentencizer(main_line).sents))
        last_sentence = main_sentences.pop()
        last_sentence = re.sub(r'\s*[:]\s*$', ' ', last_sentence)
        last_sentence = re.sub(r'\s*[.]\s*$', '. ', last_sentence) 

        merged_sentence=handle_root_list(main_sentences, last_sentence, main_line_other_line[1])
        merged_sentences+= break_long_paragraphs( merged_sentence )
        
    return merged_sentences

In [13]:
def testt1():
    print(a)

def testt():
    a=4
    testt1()
    
testt()

NameError: name 'a' is not defined

In [14]:
merge_sentences( lines )

NameError: name 'main_line' is not defined

In [16]:
merged_sentences

['ANNEX I',
 'Formulae for the determination of the liquidity buffer composition Credit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.',
 'Formulae for the determination of the liquidity buffer composition Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭.',
 'Formulae for the determination of the liquidity buffer composition ‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid asse

In [13]:
if re.search(r'^ *Formulae for', main_line.strip()): #Added by AD
    print("dsf")

dsf


In [14]:
main_sentences

['Formulae for the determination of the liquidity buffer composition, , Credit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.',
 'Formulae for the determination of the liquidity buffer composition, , Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭.',
 'Formulae for the determination of the liquidity buffer composition, , ‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets,

In [15]:
merged_sentences

['ANNEX I',
 'Formulae for the determination of the liquidity buffer composition, , Credit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.',
 'Formulae for the determination of the liquidity buffer composition, , Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭.',
 'Formulae for the determination of the liquidity buffer composition, , ‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 li

In [16]:
text=' ssds\n'
re.sub( r'\n$ *' , r' ' , text )

' ssds '

In [15]:
main_line_other_lines

[('ANNEX I', None),
 ('\r\n Formulae for the determination of the liquidity buffer composition\r\n',
  ['Credit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17',
   'Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭',
   '‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets, excluding level&nbsp;1 covered bonds, that would be held by the credit institution upon the unwind of an

In [17]:
merged_sentences

['ANNEX I',
 'Formulae for the determination of the liquidity buffer compositionCredit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.',
 'Formulae for the determination of the liquidity buffer compositionCalculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭.',
 'Formulae for the determination of the liquidity buffer composition‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets,

In [24]:
main_line=main_line_other_lines[1][0]

print( main_line )

main_sentences = list(map(lambda s: re.sub(r'[)](?![ ,;:.?! ()])(?!th)', r') ',s.text.rstrip()), sentencizer(main_line).sents)) 


 Formulae for the determination of the liquidity buffer composition



In [28]:
main_sentences

['\r\n Formulae for the determination of the liquidity buffer composition']

In [27]:
main_sentences

['\r\n Formulae for the determination of the liquidity buffer composition']

In [29]:
main_sentences[0]

'\r\n Formulae for the determination of the liquidity buffer composition'

In [15]:
main_sentences

['\r\n Formulae for the determination of the liquidity buffer compositionCredit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.',
 '\r\n Formulae for the determination of the liquidity buffer compositionCalculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭.',
 '\r\n Formulae for the determination of the liquidity buffer composition‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid asse

In [13]:
merged_sentences

['ANNEX I',
 'Formulae for the determination of the liquidity buffer compositionCredit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.',
 'Formulae for the determination of the liquidity buffer compositionCalculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭.',
 'Formulae for the determination of the liquidity buffer composition‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets,

In [11]:
merged_sentences

['ANNEX I',
 'Formulae for the determination of the liquidity buffer composition1.',
 'Formulae for the determination of the liquidity buffer compositionCredit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17.',
 'Formulae for the determination of the liquidity buffer composition2.',
 'Formulae for the determination of the liquidity buffer compositionCalculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭.',
 'Formulae for the determination of the liquidity buffer composition3.',
 'Formulae for the determination of the liquidity buffer com

In [31]:
other_lines

['1',
 'Credit institution shall use the formulae laid down in this Annex to determine the composition of their  liquidity buffer in accordance with Article 17',
 '2',
 'Calculation of the liquidity buffer: as of the calculation date, the liquidity buffer of the credit institution shall be equal to ❬the level 1 asset amount; plus ‖ and/or the level 2A asset amount; plus ‖ and/or the level 2B asset amount ‖ and/or minus the lesser of ‖ and/or the sum of (a), (b), and (c) ‖ and/or the ‘excess liquid assets amount’ as calculated in accordance with paragraphs 3 and 4 of this Annex❭',
 '3',
 '‘Excess liquid assets’ amount: this amount shall be comprised of the components defined herein ❬an adjusted non-covered bond level 1 assets amount, which shall be equal to the value of all level 1 liquid assets, excluding level&nbsp;1 covered bonds, that would be held by the credit institution upon the unwind of any secured funding transaction, secured lending transaction, asset exchange or collaterali

In [ ]:
# print all the input lines, after merging lists ":"
input_sentences = seekable_iterator(iter(input_sentences))
for main_line in input_sentences:
    main_line = main_line.rstrip("\n")
    try:
        if re.search(r'[:]\s*$',main_line):
            main_sentences = list(map(lambda s: re.sub(r'[)](?![ ,;:.?! ()])(?!th)', r') ',s.text.rstrip()), sentencizer(main_line).sents))
            last_sentence = main_sentences.pop()
            last_sentence = re.sub(r'\s*[:]\s*$', ' ', last_sentence)

            # gather the other lines
            other_lines = collect_sub_lines(input_sentences, separator=None)

            # only continue if there is an actual list
            if len(other_lines) == 0:
                print_break_long_paragraphs(main_line)
                continue

            # handle the merging of the list in the sentences
            content_to_print = handle_root_list(main_sentences, last_sentence, other_lines)

            print_break_long_paragraphs(content_to_print)
        
        #if no double-point in the sentence...
        elif re.match(section_regexp, main_line, re.IGNORECASE):
            next_line = input_sentences.next()
            if re.match(r'^[0-9]+[.] ', next_line):
                input_sentences.rewind()
            else:
                print_break_long_paragraphs(main_line + ' : ' + next_line.rstrip('\r\n'))
        else: #if no double-point in the sentence, nor other special rule
            print_break_long_paragraphs(main_line)
    except KeyboardInterrupt:
        raise
    except:
        print("Error in line: " + main_line, file=sys.stderr)

In [8]:
main_line_other_lines

[('Article 3', None),
 ('Determination of total amount of annual contributions to be raised', None),
 ('The total amount of annual contributions to be raised for a given financial year shall be calculated on the basis of Part I of the budget adopted by the Board for that financial year in accordance with Article 61 of Regulation (EU) No 806/2014, adjusted by the result of the budget of the latest financial year for which the final accounts have been published in accordance with Article 63(7) of that Regulation.',
  None),
 ('The Board shall determine the total amount of annual contributions so that Part I of the budget of the Board is balanced.',
  None),
 ('Article 4', None),
 ('Allocation of total amount of annual contributions', None),
 ('1.   The total amount to be raised in accordance with Article 3 shall be allocated as follows:',
  ['95 % to the following entities and groups ❬entities and groups referred to in Article 7(2) of Regulation (EU) No 806/2014 ‖ and/or entities and gro

In [13]:
sentence="1.   The total amount to be raised in accordance with Article 3 shall be allocated as described in the following list ❮95 % to the following entities and groups ❬entities and groups referred to in Article 7(2) of Regulation (EU) No 806/2014 ‖ and/or entities and groups with regard to which the Board has decided to exercise powers under Regulation (EU) No 806/2014 pursuant to Article 7(4)(b) of that Regulation ‖ and/or entities and groups in relation to which participating Member States have decided in accordance with Article 7(5) of Regulation (EU) No 806/2014 that the Board exercise powers and responsibilities conferred to it by that Regulation❭ ‖ 5 % to entities and groups referred to in Article 7(3) of Regulation (EU) No 806/2014❯"
print( sentence  )


1.   The total amount to be raised in accordance with Article 3 shall be allocated as described in the following list ❮95 % to the following entities and groups ❬entities and groups referred to in Article 7(2) of Regulation (EU) No 806/2014 ‖ and/or entities and groups with regard to which the Board has decided to exercise powers under Regulation (EU) No 806/2014 pursuant to Article 7(4)(b) of that Regulation ‖ and/or entities and groups in relation to which participating Member States have decided in accordance with Article 7(5) of Regulation (EU) No 806/2014 that the Board exercise powers and responsibilities conferred to it by that Regulation❭ ‖ 5 % to entities and groups referred to in Article 7(3) of Regulation (EU) No 806/2014❯


# END

In [76]:
main_line_other_lines

NameError: name 'main_line_other_lines' is not defined

In [15]:
sentence='(  ❬df| )  ❭ )'

sentence_other = re.sub(r'(^[^❬]+|[^❭]+$)',r'',sentence)
sentence_other

'❬df| )  ❭'

In [84]:
fold_sublist( lines[6][2]  )

'95 % to the following entities and groups: ⟨(i) ‖ and/or entities and groups referred to in Article 7(2) of Regulation (EU) No 806/2014; ‖ and/or (ii) ‖ and/or entities and groups with regard to which the Board has decided to exercise powers under Regulation (EU) No 806/2014 pursuant to Article 7(4)(b) of that Regulation; ‖ and/or (iii) ‖ and/or entities and groups in relation to which participating Member States have decided in accordance with Article 7(5) of Regulation (EU) No 806/2014 that the Board exercise powers and responsibilities conferred to it by that Regulation;⟩'

In [129]:
fold_sublist( [ 'a', 'b' , 'c'  ]   )

'a ❬b ‖ and/or c❭'

'a ❬b ‖ and/or c❭'

In [132]:
fold_sublist( [ [ 'a', 'b' , 'c'  ] ]  )

'❬a ⟨b ‖ and/or c⟩❭'

In [87]:
fold_sublist( [ [ 'a', 'b' , 'c'  ] ]  )

'⟨a ⟨b ‖ and/or c⟩⟩'

In [133]:
fold_sublist( [ 'a', [ 'a', 'b' , 'c' ]  ]  )

'a ❬a ⟨b ‖ and/or c⟩❭'

In [134]:
fold_sublist( [ 'a', [ 'a', 'b' , 'c' ]  ]  )  

'a ❬a ⟨b ‖ and/or c⟩❭'

In [135]:
fold_sublist( [ 'a', [ 'a' ]  ]   )

'a ❬a ⟨⟩❭'

In [136]:
fold_sublist( [ 'a', 'a'  ]   )

'a ❬a❭'

In [4]:
from typing import Set, List

def get_text_html(  cas: Cas, SofaID: str , tagnames : Set[str] = set( 'p')  ) -> List[str] :
    
    '''
    Given a cas, and a view (SofaID), this function selects all ValueBetweenTagType elements ( with tag.tagName in the set tagnames ), extracts the covered text, and returns the list of extracted sentences.
    '''
    
    sentences=[]
    for tag in cas.get_view( SofaID ).select( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType" ):
        if tag.tagName in set(tagnames):
            sentence=tag.get_covered_text().strip()
            if sentence:
                sentences.append( sentence )
                
    return sentences

sentences=get_text_html( cas, 'html2textView' )

In [6]:
with open(  "/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/txt/double_nested_list.txt", "w" ) as f:
    f.write( "\n".join(sentences)  )